In [2]:
!apt-get install yasm

E: Could not open lock file /var/lib/dpkg/lock - open (13: Permission denied)
E: Unable to lock the administration directory (/var/lib/dpkg/), are you root?


Install ffmpeg and libavcodec-dev (plus related stuff) from
https://launchpad.net/~mc3man/+archive/ubuntu/trusty-media

Install OpenCV using the instructions from 
http://www.pyimagesearch.com/2015/06/22/install-opencv-3-0-and-python-2-7-on-ubuntu/

In my case I had a build error related to missing CUDA libs, so the build command I used was

    cmake -D CMAKE_BUILD_TYPE=RELEASE \
            -D CMAKE_INSTALL_PREFIX=/usr/local \
            -D INSTALL_C_EXAMPLES=OFF \
            -D INSTALL_PYTHON_EXAMPLES=ON \
            -D "OPENCV_EXTRA_MODULES_PATH=../../opencv_contrib/modules" \
            -D BUILD_EXAMPLES=ON \
            -D WITH_CUDA=OFF ..



In [3]:
!pip install pytube

Cleaning up...


In [4]:
# Reset GPU
!sudo rmmod nvidia_uvm
!sudo rmmod nvidia
!sudo insmod /lib/modules/3.13.0-77-generic/kernel/drivers/video/nvidia.ko
!sudo insmod /lib/modules/3.13.0-77-generic/kernel/drivers/video/nvidia-uvm.ko
!echo "OK"

rmmod: ERROR: Module nvidia_uvm is in use
rmmod: ERROR: Module nvidia is in use by: nvidia_uvm
insmod: ERROR: could not insert module /lib/modules/3.13.0-77-generic/kernel/drivers/video/nvidia.ko: File exists
insmod: ERROR: could not insert module /lib/modules/3.13.0-77-generic/kernel/drivers/video/nvidia-uvm.ko: File exists
OK


In [5]:
!mkdir videos
!mkdir imageseries

mkdir: cannot create directory 'videos': File exists
mkdir: cannot create directory 'imageseries': File exists


In [6]:
from pytube import YouTube

# not necessary, just for demo purposes.
from pprint import pprint

yt = YouTube("https://www.youtube.com/watch?v=yim23IFJOyQ")

yt.set_filename('Chimpanzees1')
# Once set, you can see all the codec and quality options YouTube has made
# available for the perticular video by printing videos.

# pprint(yt.get_videos())

True

In [7]:
video = yt.get('mp4', '360p')

In [8]:
# In this case, we'll need to specify both the codec (mp4) and resolution
# (either 360p or 720p).

# Okay, let's download it!
video = yt.get('mp4', '360p')
#video.download('./videos/')


In [9]:
# Now let's try to read this using our fresh opencv build..

In [1]:
import cv2
import h5py


In [2]:
from IPython.display import HTML, FileLink, display
from base64 import b64encode
def link_video(path):
    src = FileLink(path)
    video_tag = '<video controls alt="test" src="%s">' % (path)
    
    display(HTML(data=video_tag))


In [3]:
vc = cv2.VideoCapture('videos/Chimpanzees1.mp4')
frames = 0
shape = None
dtype = None
while (True):
    ret, img = vc.read()
    # img should be an numpy ndarray of shape (width, heigth, channels)
    if (not ret):
        break
    if (shape is None):
        shape = img.shape
    if (dtype is None):
        dtype = img.dtype
    frames+=1
    
print "Could read %d frames of shape %r and dtype %r" % (frames, shape, dtype)

Could read 1490 frames of shape (360, 640, 3) and dtype dtype('uint8')


In [4]:
import h5py

h5f = h5py.File('/tmp/Chimpanzees1_6.hdf5',driver='core', backing_store=False)
h5g = h5f.create_group('input')

In [5]:
import numpy as np

In [6]:
ishape = tuple([ frames ] + list(shape))
ichunkshape = tuple([ 1 ] + list(shape))
imaxshape = tuple([ None ] + list(shape))

In [7]:
import numpy as np
from numpy.random import permutation


In [8]:
inframes = h5g.create_dataset('raw_hsl_frames', 
                             shape=ishape, dtype=np.float32, 
                             chunks=ichunkshape, maxshape=imaxshape,
                             fletcher32=True, fillvalue=0)


In [18]:
!pip freeze |grep h5py

h5py==2.2.1


In [9]:
permuted_indices = permutation(frames)

In [10]:
print shape

(360, 640, 3)


In [11]:
vc = cv2.VideoCapture('videos/Chimpanzees1.mp4')
frames = 0
shape = None
dtype = None
while (True):
    ret, img = vc.read()
    if (not ret):
        break
    hsvimg = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #print hsvimg.dtype
    
    hsvimgf = hsvimg.astype(np.float32) / 255.0
    # img should be an numpy ndarray of shape (width, heigth, channels)
    inframes[permuted_indices[frames],:,:,:] = hsvimgf
    
    hsvimg2 = (inframes[permuted_indices[frames],:,:,:] * 255.0).astype(np.uint8)
    
    #print np.sum(hsvimg-hsvimg2)
    #bgrimg = cv2.cvtColor(hsvimg2, cv2.COLOR_HSV2BGR)
    #print np.sum(img-bgrimg)
    
    #cv2.imwrite("videos/recoded1_%05d.png" % (frames), bgrimg)
    
    frames+=1
    if (frames % 50==0):
        print "Frame #%d" % (frames)
print "Read %d frames" % (frames)


Frame #50
Frame #100
Frame #150
Frame #200
Frame #250
Frame #300
Frame #350
Frame #400
Frame #450
Frame #500
Frame #550
Frame #600
Frame #650
Frame #700
Frame #750
Frame #800
Frame #850
Frame #900
Frame #950
Frame #1000
Frame #1050
Frame #1100
Frame #1150
Frame #1200
Frame #1250
Frame #1300
Frame #1350
Frame #1400
Frame #1450
Read 1490 frames


In [12]:
ls -hs videos/Chimpanzees1.hdf5

ls: cannot access videos/Chimpanzees1.hdf5: No such file or directory


In [13]:
import numpy as np

In [14]:
!rm videos/imageseries*.png
#!ffmpeg -i videos/Chimpanzees1.mp4 -r 6 -f image2 imageseries/Chimpanzees1_%05d.png

rm: cannot remove ‘videos/imageseries*.png’: No such file or directory


In [15]:
!rm videos/*.png
!ls videos

rm: cannot remove ‘videos/*.png’: No such file or directory
Capoeira_Girl.mp4   Chimpanzees1_reconstructed2.mp4  Solar-System.mp4
Capoeira_Movie.mp4  Chimpanzees1_reconstructed3.mp4
Chimpanzees1.mp4    Chimpanzees.mp4


In [26]:
#!rm videos/Chimpanzees1_recoded.mp4
#!ffmpeg -i imageseries/Chimpanzees1_%05d.png -c:v libx264 -b:v 400k -vf format=yuv420p -maxrate:v 500k -bufsize 1000k  -r 10 -preset medium videos/Chimpanzees1_recoded.mp4

In [17]:
!ls -lahs videos/Chimpanzees1*.mp4

5.3M -rw-rw-r-- 1 nimbix nimbix 5.3M Apr 27 17:54 videos/Chimpanzees1.mp4
1.6M -rw-rw-r-- 1 nimbix nimbix 1.6M Apr 29 10:40 videos/Chimpanzees1_reconstructed2.mp4
100K -rw-rw-r-- 1 nimbix nimbix 100K May 11 18:05 videos/Chimpanzees1_reconstructed3.mp4


In [28]:
#link_video('videos/Chimpanzees1_recoded.mp4')

In [29]:
import os
import h5py
from glob import glob



In [19]:
!cat ~/.keras/keras.json

{"epsilon": 1e-07, "floatx": "float32", "backend": "tensorflow"}



Note:

Read http://arxiv.org/pdf/1402.3337.pdf and use the ThresholdedReLu activation from http://keras.io/layers/advanced_activations/


In [20]:

import numpy as np
np.random.seed(1337)  # for reproducibility


from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers.advanced_activations import ThresholdedReLU
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution2D, ZeroPadding2D
from keras.datasets import imdb
from keras.optimizers import SGD
from keras.regularizers import l1,l2,l1l2, ActivityRegularizer, activity_l1
from keras import backend as K

Using TensorFlow backend.


In [21]:
import keras.layers.convolutional as kconv

In [22]:
print 'Build model...X'

num_feature_detectors=12*12*3
tile_size=12
receptive_field_size=16
model = Sequential()
model.add(Dropout(0.001, input_shape=ichunkshape[1:]))

input_features = Convolution2D( name='encode_tiled_features', 
                        nb_filter=num_feature_detectors, nb_row=receptive_field_size,nb_col=receptive_field_size, 
                        subsample=(tile_size,tile_size), 
                        border_mode='valid', trainable=True,
                        dim_ordering='tf' )


print np.prod(ichunkshape[1:])
model.add(input_features)

print input_features.output_shape 

model.add(Activation('relu'))

decoder = Convolution2D( name='decode_tiled_features', 
                        nb_filter=tile_size*tile_size*3, nb_row=1,nb_col=1, subsample=(1,1),
                        border_mode='same', trainable=True,
                        dim_ordering='tf')

model.add(decoder)
model.add(Reshape(tuple(list(decoder.output_shape[1:3]) + [tile_size, tile_size, 3])))
print model.output_shape
model.add(Permute((1,3,2,4,5)))

print model.output_shape
model.add(Reshape((model.output_shape[1]*model.output_shape[2],model.output_shape[3]*model.output_shape[4],model.output_shape[5])))

model.add(ZeroPadding2D(((ichunkshape[1]-model.output_shape[1])/2, (ichunkshape[2]-model.output_shape[2])/2), 
                        dim_ordering='tf'))

print model.output_shape
def mean_abs_error_all(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis=None)



print model.output_shape

Build model...X
691200
(None, 29, 53, 432)
(None, 29, 53, 12, 12, 3)
(None, 29, 12, 53, 12, 3)
(None, 360, 640, 3)
(None, 360, 640, 3)


In [23]:
import tensorflow as tf

In [24]:
model.compile(optimizer='adam', loss=mean_abs_error_all, class_mode='binary')

/usr/local/lib/python2.7/dist-packages/keras/models.py:332: UserWarning: "class_mode" argument is deprecated, please remove it.
  warnings.warn('"class_mode" argument is deprecated, '


In [25]:
inframesmem = np.array(inframes)

In [ ]:
#h5f.close()

In [ ]:
def data_generator(batchsize):
    for i in range(10):
        for j in permutation(1300):
            bi = inframesmem[j:(j+batchsize),:,:,:]

            yield bi,  bi

for i in range(30):
    print "Over-Epoch %i" % (i)
    model.fit_generator(data_generator(40), validation_data=(inframes[1400:1430], inframes[1400:1430]),
              show_accuracy=False, nb_epoch=1, samples_per_epoch=500, nb_worker=1)

Over-Epoch 0
Epoch 1/1
520/500 [===============================] - 4s - loss: 0.0446 - val_loss: 0.0460
Over-Epoch 1
Epoch 1/1
520/500 [===============================] - 4s - loss: 0.0447 - val_loss: 0.0436
Over-Epoch 2
Epoch 1/1
360/500 [====================>.........] - ETA: 1s - loss: 0.0438

In [ ]:
model.save_weights('model1.hdf5')

In [33]:
inframesm = np.array(inframes)

In [26]:
type(inframesm)

NameError: name 'inframesm' is not defined

In [ ]:
def data_generator(batchsize):
    for i in range(10):
        for j in permutation(1300):
            bi = inframesm[j:(j+batchsize),:,:,:]
            yield bi,  bi

for i in range(1):
    print "Over-Epoch %i" % (i)
    model.fit_generator(data_generator(4), validation_data=(inframes[1400:1430], inframes[1400:1430]),
              show_accuracy=False, nb_epoch=5, samples_per_epoch=400, nb_worker=1)

In [ ]:
K.floatx()

In [34]:
import tensorflow as tf
from keras.layers import MaskedLayer
import keras.backend as K

_FLOATX = K.floatx()



In [35]:
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization

In [ ]:
print 'Build model...X'

num_feature_detectors=16*16*4
tile_size=16
receptive_field_size=24
model = Sequential()
model.add(Dropout(0.1, input_shape=ichunkshape[1:]))

input_features = Convolution2D( name='encode_tiled_features', 
                        nb_filter=num_feature_detectors, nb_row=receptive_field_size,nb_col=receptive_field_size, 
                        subsample=(tile_size,tile_size), 
                        border_mode='valid', trainable=True,
                        dim_ordering='tf', activity_regularizer=activity_l1(0.0000005))


model.add(input_features)
model.add(PReLU())

decoder = Convolution2D( name='decode_tiled_features', 
                        nb_filter=tile_size*tile_size*3, nb_row=1,nb_col=1, subsample=(1,1),
                        border_mode='same', trainable=True,
                        dim_ordering='tf')

model.add(decoder)
model.add(Reshape(tuple(list(decoder.output_shape[1:3]) + [tile_size, tile_size, 3])))
print model.output_shape
model.add(Permute((1,3,2,4,5)))

print model.output_shape
model.add(Reshape((model.output_shape[1]*model.output_shape[2],model.output_shape[3]*model.output_shape[4],model.output_shape[5])))

model.add(ZeroPadding2D(((ichunkshape[1]-model.output_shape[1])/2, (ichunkshape[2]-model.output_shape[2])/2), 
                        dim_ordering='tf'))

print model.output_shape
def mean_abs_error_all(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis=None)



print model.output_shape

In [ ]:
model.compile(optimizer='adam', loss=mean_abs_error_all, class_mode='binary')


In [ ]:
def data_generator(batchsize):
    for i in range(10):
        perm = permutation(1300)
        for i in range(len(perm)//batchsize):
            bi = inframesmem[perm[i*batchsize:(i+1)*batchsize],:,:,:]
            yield bi,  bi

for i in range(1):
    print "Over-Epoch %i" % (i)
    model.fit_generator(data_generator(30), validation_data=(inframes[1400:1430], inframes[1400:1430]),
              show_accuracy=False, nb_epoch=5, samples_per_epoch=400, nb_worker=1)

In [36]:
print 'Build model...X'

num_feature_detectors=12*12*3
tile_size=12
receptive_field_size=16
model = Sequential()
model.add(Dropout(0.1, input_shape=ichunkshape[1:]))

enc1 = Convolution2D( name='enc1', 
                        nb_filter=num_feature_detectors, nb_row=receptive_field_size,nb_col=receptive_field_size, 
                        subsample=(tile_size,tile_size), 
                        border_mode='valid', trainable=True,
                        dim_ordering='tf', activity_regularizer=activity_l1(0.0000005))


model.add(enc1)
model.add(PReLU())
model.add(Dropout(0.1))
# ---------------
tile_size2=2
num_feature_detectors2=tile_size2*tile_size2*num_feature_detectors 
receptive_field_size2=2

enc2 = Convolution2D( name='enc2', 
                        nb_filter=num_feature_detectors2, nb_row=receptive_field_size2,nb_col=receptive_field_size2, 
                        subsample=(tile_size2,tile_size2), 
                        border_mode='valid', trainable=True,
                        dim_ordering='tf', activity_regularizer=activity_l1(0.0000005))


model.add(enc2)
model.add(PReLU())

# --------------------

dec2 = Convolution2D( name='dec2', 
                        nb_filter=tile_size2*tile_size2*num_feature_detectors, nb_row=1,nb_col=1, subsample=(1,1),
                        border_mode='valid', trainable=True,
                        dim_ordering='tf')

model.add(dec2)
# We need to reshape and reorder dimensions, so we can, at least in theory, do symmetric, layer-wise training...
newshape2 = tuple(list(dec2.output_shape[1:3])+ [tile_size2, tile_size2, num_feature_detectors])
model.add(Reshape(newshape2))
model.add(Permute((1,3,2,4,5)))
blockshape2 = (model.output_shape[1]*model.output_shape[2], model.output_shape[3]*model.output_shape[4], model.output_shape[5])

model.add(Reshape(blockshape2))

# --------------------

dec1 = Convolution2D( name='dec1', 
                        nb_filter=tile_size*tile_size*3, nb_row=1,nb_col=1, subsample=(1,1),
                        border_mode='same', trainable=True,
                        dim_ordering='tf')
model.add(dec1)
newshape1 = tuple(list(dec1.output_shape[1:3])+ [tile_size, tile_size, 3])
model.add(Reshape(newshape1))
model.add(Permute((1,3,2,4,5)))

blockshape1 = (model.output_shape[1]*model.output_shape[2],model.output_shape[3]*model.output_shape[4], model.output_shape[5])

model.add(Reshape(blockshape1))

model.add(ZeroPadding2D(((ichunkshape[1]-model.output_shape[1])/2, (ichunkshape[2]-model.output_shape[2])/2), 
                        dim_ordering='tf'))

# --------------------

print model.output_shape
def mean_abs_error_all(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis=None)



print model.output_shape

Build model...X
(None, 360, 640, 3)
(None, 360, 640, 3)


In [37]:
model.compile(optimizer='adam', loss=mean_abs_error_all, class_mode='binary')

RuntimeError: ('The following error happened while compiling the node', GpuDnnConv{algo='small', inplace=True}(GpuContiguous.0, GpuContiguous.0, GpuAllocEmpty.0, GpuDnnConvDesc{border_mode='valid', subsample=(12, 12), conv_mode='conv', precision='float32'}.0, Constant{1.0}, Constant{0.0}), '\n', 'could not create cuDNN handle: CUDNN_STATUS_INTERNAL_ERROR', "[GpuDnnConv{algo='small', inplace=True}(<CudaNdarrayType(float32, 4D)>, <CudaNdarrayType(float32, 4D)>, <CudaNdarrayType(float32, 4D)>, <CDataType{cudnnConvolutionDescriptor_t}>, Constant{1.0}, Constant{0.0})]")

In [1]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

Couldn't import dot_parser, loading of dot files will not be possible.
[GpuElemwise{exp,no_inplace}(<CudaNdarrayType(float32, vector)>), HostFromGpu(GpuElemwise{exp,no_inplace}.0)]
Looping 1000 times took 0.453555 seconds
Result is [ 1.23178029  1.61879349  1.52278066 ...,  2.20771813  2.29967761
  1.62323296]
Used the gpu


Using gpu device 0: GRID K520 (CNMeM is enabled)


In [ ]:
def data_generator(batchsize):
    while True:
        perm = permutation(1400)
        for i in range(len(perm)//batchsize):
            bi = inframesmem[perm[i*batchsize:(i+1)*batchsize],:,:,:]
            yield bi,  bi

for i in range(1):
    print "Over-Epoch %i" % (i)
    model.fit_generator(data_generator(40), validation_data=(inframes[1400:1480], inframes[1400:1480]),
              show_accuracy=False, nb_epoch=10, samples_per_epoch=500, nb_worker=1)

In [ ]:
model.save_weights('sparse_autoencoder_4layers.hdf5')

In [ ]:
import cv2

In [ ]:
!rm imageseries/*.jpg

In [ ]:
j = 0
c = 0
for i in permuted_indices:
    
    
    #print np.sum(hsvimg-hsvimg2)
    fframe = inframes[i,:,:,:]
    frame = (fframe * 255.0).astype(np.uint8)
    
    bgrimg = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
    
    pframe = model.predict(fframe[np.newaxis,:,:,:], batch_size=1)
    np.clip(pframe, 0.0, 1.0, pframe)
    #print mean_squared_error_all
    piframe = (pframe[0,:,:,:]*255.0).astype(np.uint8)
    rgbimg = cv2.cvtColor(piframe, cv2.COLOR_HSV2BGR)
    
    j+=1
    c+=1
    cv2.imwrite("imageseries/reconstruction1_%05d.jpg" % (c), rgbimg)
    
    if (j==800):
        for z in range(200):
            pframe = model.predict(pframe, batch_size=1)
            np.clip(pframe, 0.0, 1.0, pframe)
            #print mean_squared_error_all
            piframe = (pframe[0,:,:,:]*255.0).astype(np.uint8)
            rgbimg = cv2.cvtColor(piframe, cv2.COLOR_HSV2BGR)
            c+=1
            cv2.imwrite("imageseries/reconstruction1_%05d.jpg" % (c), rgbimg)
    if (j % 100==0):
        me = np.mean(np.square((bgrimg.astype(np.float32)/255.0)-(rgbimg.astype(np.float32)/255.0)))
        print "Wrote image %d - MSE: %f" % (j, me)

In [ ]:
!rm imageseries/*.png

In [ ]:
!rm videos/Chimpanzees1_reconstructed2.mp4
!ffmpeg -i imageseries/reconstruction1_%05d.jpg -c:v libx264 -b:v 400k -maxrate:v 500k -bufsize 1000k -r 10 -preset medium videos/Chimpanzees1_reconstructed2.mp4


In [ ]:
!ls -hs videos/*.mp4

In [ ]:
link_video('videos/Chimpanzees1_reconstructed2.mp4')
#link_video('videos/Chimpanzees1_reconstructed2.mp4')

In [ ]:
display(FileLink('videos/Chimpanzees1_reconstructed2.mp4'))

In [ ]:
class array(object) :
    """Simple Array object that support autodiff."""
    def __init__(self, value, name=None):
        self.value = value
        if name:
            self.grad = lambda g : {name : g}

    def __add__(self, other):
        assert isinstance(other, int)
        ret = array(self.value + other)
        ret.grad = lambda g : self.grad(g)
        return ret

    def __mul__(self, other):
        assert isinstance(other, array)
        ret = array(self.value * other.value)
        def grad(g):
            x = self.grad(g * other.value)
            x.update(other.grad(g * self.value))
            return x
        ret.grad = grad
        return ret

# some examples
a = array(1, 'a')
b = array(2, 'b')
c = b * a
d = c + 1
print d.value
print d.grad(1)

In [ ]:
(b*a).grad(2)